In [ ]:
!pip install autokeras
!pip install tweet-preprocessor
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 32.1 MB/s 
     |████████████████████████████████| 135 kB 75.3 MB/s 
     |████████████████████████████████| 1.6 MB 70.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 32.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak

In [ ]:
!gdown 1U6CU4VonapD40kAkK3_X6LirG1_f8nim
!gdown 1gBqHkxh5Bg4O_VIEzrLO-mG9tkZ8uD_F

Downloading...
From: https://drive.google.com/uc?id=1U6CU4VonapD40kAkK3_X6LirG1_f8nim
To: /content/H1_Offensive_Language_Identification_test.csv
100% 132k/132k [00:00<00:00, 67.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gBqHkxh5Bg4O_VIEzrLO-mG9tkZ8uD_F
To: /content/H1_Offensive_Language_Identification_train.csv
100% 1.83M/1.83M [00:00<00:00, 205MB/s]


In [ ]:
tweets_df_train = pd.read_csv("/content/H1_Offensive_Language_Identification_train.csv")
tweets_df_test = pd.read_csv("/content/H1_Offensive_Language_Identification_test.csv")

In [ ]:
import preprocessor as p
import numpy as np

# Remove numbers, emojis and &'s
p.set_options(p.OPT.NUMBER, p.OPT.EMOJI)

tweets_df_train_cleaned = (tweets_df_train.assign(tweet=tweets_df_train["tweet"].apply(p.clean).str.replace("&", "and ").str[:512],label=np.where(tweets_df_train.label == "OFF", 0, 1) )) # Change OFF to 1 and NOT to 0))


tweets_df_test_cleaned = (tweets_df_test.assign(tweet=tweets_df_test["tweet"].apply(p.clean).str.replace("&", "and ").str[:512],))

In [ ]:
import re
import nltk
import string
import pandas as pd
from textblob import Word
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove("not")
stop_words.remove("no")
pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')

html = re.compile('<.*?>')
spell = SpellChecker()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def preprocess(text, stop_word_pattern = pattern, html_pattern = html):
    text = re.sub("([@][A-Za-z0-9_]+)|(\w+:\/\/\S+)", "", text)
    text = re.sub("\s+"," ",text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    text = re.sub(html_pattern, '', text)
    text = re.sub(r"URL", '', text)
    text = re.sub("\s+"," ",text)
    return text.lower()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
cleaned = [preprocess(sent) for sent in tweets_df_train_cleaned["tweet"]]
tweets_df_train_cleaned["tweet"] = cleaned

In [ ]:
x_train = np.array(tweets_df_train_cleaned.tweet)
y_train = np.array(tweets_df_train_cleaned.label)

In [ ]:
clf = ak.TextClassifier(max_trials=5)
clf.fit(x_train, y_train,validation_split = 0.065, epochs=3)

Trial 5 Complete [00h 10m 19s]
val_loss: 0.434757798910141

Best val_loss So Far: 0.434757798910141
Total elapsed time: 00h 31m 53s
Epoch 1/3
414/414 [==============================] - 217s 494ms/step - loss: 0.4910 - accuracy: 0.7656
Epoch 2/3
414/414 [==============================] - 204s 493ms/step - loss: 0.3769 - accuracy: 0.8372
Epoch 3/3
414/414 [==============================] - 204s 493ms/step - loss: 0.3006 - accuracy: 0.8779


In [ ]:
# get the best performing model
loaded_model = clf.export_model()

# display the architecture of the best performing model
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 expand_last_dim (ExpandLastDim  (None, 1)           0           ['input_1[0][0]']                
 )                                                                                                
                                                                                                  
 bert_tokenizer (BertTokenizer)  ((None, None),      0           ['expand_last_dim[0][0]']        
                                 (None, None),                                                    
                                 (None, None))                                                

In [ ]:
from tensorflow.keras.models import load_model
# save the model on disk
loaded_model.save("best_model", save_format = "tf")

# load the model from disk
loaded_model = load_model("best_model",custom_objects = ak.CUSTOM_OBJECTS)

In [ ]:
!zip -r /content/best_model.zip /content/best_model

  adding: content/best_model/ (stored 0%)
  adding: content/best_model/keras_metadata.pb (deflated 95%)
  adding: content/best_model/variables/ (stored 0%)
  adding: content/best_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/best_model/variables/variables.index (deflated 76%)
  adding: content/best_model/assets/ (stored 0%)
  adding: content/best_model/saved_model.pb (deflated 93%)


In [ ]:
from google.colab import files
files.download("/content/best_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
cleaned = [preprocess(sent) for sent in tweets_df_test_cleaned["tweet"]]
tweets_df_test_cleaned["tweet"] = cleaned
x_test = np.array(tweets_df_test_cleaned.tweet)

# use loaded model for future predictions on new samples
y_hat = loaded_model.predict(x_test)

27/27 [==============================] - 6s 176ms/step


In [ ]:
y = list(np.where(y_hat.reshape(-1) <=0.6, 0 , 1))
y

In [ ]:
df = pd.DataFrame()
df["label"] = y
df["id"] = tweets_df_test.id
df.to_csv("final.csv")